In [1]:
#This is an implementation of the first algorithm shown in the paper : clustering with generalized eigen-decomposition.

In [2]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Layers are sorted by relevance
nodes = np.array([1,2,3,4,5,6,7,8])
W = np.array([
    [[1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,0,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1]],

    [[0,0,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,1,1]],

    [[0,0,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,0,0,0,0],
     [1,1,1,1,1,0,0,0],
     [0,0,0,1,1,1,1,1],
     [0,0,0,0,1,1,1,1],
     [0,0,0,0,1,1,0,0],
     [0,0,0,0,1,1,1,1]]  ])

In [4]:
def compute_degree_matrix(M):
  res = np.array([0]*len(M))
  for i in range(len(M)):
    for j in range(len(M)):
      if(M[i][j] != 0):
        res[i]+=1
  return np.diag(res)

In [5]:
def k_means_cluster(U,k):
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(U)
  return kmeans.labels_

In [14]:
#alpha and beta have to be choosen according to the dataset we are studying


def spectral_clustering(W,k):


  alpha = 10
  beta = 100
  iter = 0
  iter_max = 1000
  stopping_condition = False
  condition = 0.001

  M,n,l = W.shape
  L = np.zeros(np.shape(W))
  delta = np.zeros(np.shape(W))
  for i in range(M):
    D = compute_degree_matrix(W[i])
    L[i] = np.linalg.inv(D)@(D-W[i])
    eigenvalues,eigenvectors = np.linalg.eig(L[i])
    delta[i] = np.diag(eigenvalues)

  eigenvalues,eigenvectors = np.linalg.eig(L[0])
  P = eigenvectors
  Q = np.linalg.inv(P)

  while(stopping_condition == False and iter<iter_max):

    print("iter : ",iter)

    def function_to_optimize_P(P_vectorized):
      P = P_vectorized.reshape((n,n))
      res = 0.5*alpha*(np.linalg.norm(P,'fro')**2 + np.linalg.norm(Q,'fro')**2) + 0.5*beta*np.linalg.norm(P@Q-np.identity(n),'fro')**2
      for i in range(M):
        res += 0.5*np.linalg.norm(L[i] - P@delta[i]@Q,'fro')**2
      return res

    def function_to_optimize_P_gradient(P_vectorized):
      P = P_vectorized.reshape((n,n))
      res = alpha*P + beta*(P@Q - np.identity(n))@np.transpose(Q)
      for i in range(M):
        res = res - (L[i] - P@delta[i]@Q)@np.transpose(Q)@delta[i]
      return res.flatten()


    x0 = P.flatten()
    val_init = function_to_optimize_P(x0)
    if(iter == 0):
      print("Valeur initiale : ", val_init)
    result_optimization_P = minimize(function_to_optimize_P, x0,method='L-BFGS-B',jac = function_to_optimize_P_gradient)
    P = result_optimization_P.x.reshape((n,n))

    print("Optimisation sur P, résultat : ", result_optimization_P.success , ", message : ", result_optimization_P.message)
    print(result_optimization_P.fun)


    def function_to_optimize_Q(Q_vectorized):
      Q = Q_vectorized.reshape((n,n))
      res = 0.5*alpha*( np.linalg.norm(P,'fro')**2 + np.linalg.norm(Q,'fro')**2 ) + 0.5*beta*np.linalg.norm(P@Q-np.identity(n),'fro')**2
      for i in range(M):
        res += 0.5*np.linalg.norm(L[i] - P@delta[i]@Q,'fro')**2
      return res

    def function_to_optimize_Q_gradient(Q_vectorized):
      Q = Q_vectorized.reshape((n,n))
      res = alpha*Q + beta*(P@Q - np.identity(n))@P
      for i in range(M):
        res = res - (L[i] - P@delta[i]@Q)@P@delta[i]
      return res.flatten()

    x0 = Q.flatten()
    result_optimization_Q = minimize(function_to_optimize_Q, x0,method='L-BFGS-B') #,  jac = function_to_optimize_Q_gradient)
    Q = result_optimization_Q.x.reshape((n,n))
    val_end = result_optimization_Q.fun

    print("Optimisation sur Q, résultat : ", result_optimization_Q.success , ", message : ", result_optimization_Q.message)
    print(result_optimization_Q.fun)


    if(val_init - val_end < condition):
      stopping_condition = True

    iter += 1

  U = P
  U = U[:,:k]
  return k_means_cluster(U,k)

In [15]:
spectral_clustering(W,2)
# there is a problem with the gradient that I dont know how to solve

iter :  0
Valeur initiale :  113.60147219597448
Optimisation sur P, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
108.627962674134
Optimisation sur Q, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
93.83191717259439
iter :  1
Optimisation sur P, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
85.16890852502941
Optimisation sur Q, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
81.9904061089562
iter :  2
Optimisation sur P, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
79.86463134984456
Optimisation sur Q, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
78.70337619233297
iter :  3
Optimisation sur P, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
77.92121814457082
Optimisation sur Q, résultat :  True , message :  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
77.45407617313

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, 0, 1, 1, 1, 1], dtype=int32)

In [ ]:
#variant : we solve the total problem with respect to both variables

#alpha and beta have to be choosen according to the dataset we are studying

alpha = 10
beta = 100

def spectral_clustering_variant(W,k):
  M,n,l = W.shape
  L = np.zeros(np.shape(W))
  for i in range(len(W)):
    D = compute_degree_matrix(W[i])
    L[i] = np.linalg.inv(D)@(D-W[i])

  def function_to_optimize(vector):
    P_vectorized = vector[:n*n]
    Q_vectorized = vector[n*n:]
    P = P_vectorized.reshape((n,n))
    Q = Q_vectorized.reshape((n,n))
    res = 0.5*alpha*(np.linalg.norm(P,'fro') + np.linalg.norm(Q,'fro')) + 0.5*beta*np.linalg.norm(P@Q-np.identity(n),'fro')
    for i in range(M):
      eigenvalues,eigenvectors = np.linalg.eig(L[i])
      delta = np.diag(eigenvalues)
      res += 0.5*np.linalg.norm(L[i] - P@delta@Q,'fro')
    return res

  eigenvalues,eigenvectors = np.linalg.eig(L[0])
  x0 = np.concatenate((eigenvectors.flatten(),np.linalg.inv(eigenvectors).flatten()))
  result_optimization = minimize(function_to_optimize, x0,method='L-BFGS-B')

  print("Optimisation réusssie : ", result_optimization.success)
  print(result_optimization.message)

  result_optimization = result_optimization.x

  U = result_optimization[:n*n].reshape((n,n))
  U = U[:,:k]
  return k_means_cluster(U,k)

In [ ]:
a = np.array([[1,2],[3,4]])
eigenvalues,eigenvectors = np.linalg.eig(a)
print(np.diag(eigenvalues))




[[-0.37228132  0.        ]
 [ 0.          5.37228132]]
